<a href="https://colab.research.google.com/github/C23-PS396/LeftLovers-MachineLearning/blob/main/Food_Recommendation_(Experimental).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# kaggle API
# setting up to use kaggle API
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d ahmedshahriarsakib/uber-eats-usa-restaurants-menus

uber-eats-usa-restaurants-menus.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/uber-eats-usa-restaurants-menus.zip

Archive:  /content/uber-eats-usa-restaurants-menus.zip
replace restaurant-menus.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: restaurant-menus.csv    A

  inflating: restaurants.csv         


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


In [ ]:
# load the dataset
df = pd.read_csv('/content/restaurants.csv')
df.head()

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703
1,2,9,J' ti`'z Smoothie-N-Coffee Bar,NaN,NaN,"Coffee and Tea, Breakfast and Brunch, Bubble Tea",NaN,"1521 Pinson Valley Parkway, Birmingham, AL, 35217",35217,33.583640,-86.773330
2,3,6,Philly Fresh Cheesesteaks (541-B Graymont Ave),NaN,NaN,"American, Cheesesteak, Sandwiches, Alcohol",$,"541-B Graymont Ave, Birmingham, AL, 35204",35204,33.509800,-86.854640
3,4,17,Papa Murphy's (1580 Montgomery Highway),NaN,NaN,Pizza,$,"1580 Montgomery Highway, Hoover, AL, 35226",35226,33.404439,-86.806614
4,5,162,Nelson Brothers Cafe (17th St N),4.7,22.0,"Breakfast and Brunch, Burgers, Sandwiches",NaN,"314 17th St N, Birmingham, AL, 35203",35203,33.514730,-86.811700


In [ ]:
# EDA and clean up
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40227 entries, 0 to 40226
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            40227 non-null  int64  
 1   position      40227 non-null  int64  
 2   name          40227 non-null  object 
 3   score         22254 non-null  float64
 4   ratings       22254 non-null  float64
 5   category      40204 non-null  object 
 6   price_range   33581 non-null  object 
 7   full_address  39949 non-null  object 
 8   zip_code      39940 non-null  object 
 9   lat           40227 non-null  float64
 10  lng           40227 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 3.4+ MB


In [ ]:
menu_df = pd.read_csv('/content/restaurant-menus.csv')
menu_df.head()

,restaurant_id,category,name,description,price
0,1,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
1,1,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
2,1,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
3,1,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
4,1,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD


In [ ]:
joined_df = df.join(menu_df.set_index('restaurant_id'), on='id',rsuffix='menu_')
joined_df.head()

,id,position,name,score,ratings,category,price_range,full_address,zip_code,lat,lng,categorymenu_,namemenu_,description,price
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99 USD
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99 USD
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99 USD
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99 USD
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches",$,"224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99 USD


In [ ]:
# remove price range column
joined_df.drop(columns=['price_range'], axis=1, inplace=True)

In [ ]:
joined_df.columns

Index(['id', 'position', 'name', 'score', 'ratings', 'category',
       'full_address', 'zip_code', 'lat', 'lng', 'categorymenu_', 'namemenu_',
       'description', 'price'],
      dtype='object')

In [ ]:
joined_df['price'] = joined_df['price'].str.replace(" USD", "")

In [ ]:
joined_df['price'] = joined_df['price'].astype(np.float64)

In [ ]:
#converting price col to float
avg_price_df = joined_df[['id', 'name', 'price']].groupby(['id', 'name']).mean('price').reset_index().rename(columns = {'price':'price_avg_usd'})

In [ ]:
joined_df['avg_price_usd'] = avg_price_df['price_avg_usd']
joined_df.head()

,id,position,name,score,ratings,category,full_address,zip_code,lat,lng,categorymenu_,namemenu_,description,price,avg_price_usd
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches","224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large Meat Lovers,Whole pie.,15.99,5.663684
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches","224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large Supreme,Whole pie.,15.99,5.663684
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches","224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large Pepperoni,Whole pie.,14.99,5.663684
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches","224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large BBQ Chicken &amp; Bacon,Whole Pie,15.99,5.663684
0,1,19,PJ Fresh (224 Daniel Payne Drive),NaN,NaN,"Burgers, American, Sandwiches","224 Daniel Payne Drive, Birmingham, AL, 35207",35207,33.562365,-86.830703,Extra Large Pizza,Extra Large 5 Cheese,Whole pie.,14.99,5.663684


In [ ]:
# drop col
cols_drop = ['score', 'ratings', 'full_address', 'zip_code', 'categorymenu_', 'namemenu_',
       'description', 'price', 'position']
joined_df.drop(columns=cols_drop, axis=1, inplace=True)

In [ ]:
joined_df.columns

Index(['id', 'name', 'category', 'lat', 'lng', 'avg_price_usd'], dtype='object')

In [ ]:
joined_df = joined_df.drop_duplicates()

In [ ]:
joined_df.shape

(40227, 6)

In [ ]:
# get all categories
joined_df['category'] = joined_df['category'].fillna('None')
temp = joined_df['category'].str.split(', ').tolist()
features = set()
for cats in temp:
  for cat in cats:
    t = cat.lower()
    features.add(t)

print("top unique features features:")
features

<ipython-input-96-f162c70d2a11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df['category'] = joined_df['category'].fillna('None')


top unique features features:


{'aapi-owned',
 'adult',
 'affordable meals',
 'afghan',
 'african',
 'african: ethiopian',
 'african: other',
 'alcohol',
 'allergy friendly',
 'ameican',
 'american',
 'american (new)',
 'american (traditional)',
 'arabian',
 'arepa',
 'argentinian',
 'armenian',
 'asado',
 'asian',
 'asian cuisine',
 'asian fusion',
 'asian-owned',
 'asian: other',
 'assorted stores',
 'aus burger',
 'australian',
 'açaí',
 'baby',
 'bacon',
 'bagels',
 'bakery',
 'balkan',
 'bangladeshi',
 'bar / pub food',
 'bar food',
 'barfood',
 'bbq',
 'beauty supply',
 'beef noodles',
 'beer',
 'belgian',
 'bento',
 'biryani',
 'bistro',
 'black-owned',
 'bogo',
 'bolivian',
 'bookstore',
 'bowls',
 'braised dishes',
 'braised pork rice',
 'brazilian',
 'breakfast',
 'breakfast &amp; brunch',
 'breakfast and brunch',
 'british',
 'brunch',
 'bubble tea',
 'burger',
 'burgers',
 'burmese',
 'burritos',
 'butchershop',
 'cafe',
 'cajun',
 'cajun / creole',
 'cakes',
 'cambodian',
 'canadian',
 'candy',
 'canton

In [ ]:
rand_ind = np.random.randint(0, len(list(features)), size=20)
features = list(features)
new_20_features = [features[i] for i in rand_ind]
new_20_features

['flour based food',
 'belgian',
 'exclusive to eats',
 'burritos',
 'sopas e caldos',
 'local eats',
 'bangladeshi',
 'russian',
 'japanese sweets',
 'australian',
 'dessert: other',
 'healthy',
 'swedish',
 'middle eastern',
 'chinese: taiwanese',
 'american (traditional)',
 'balkan',
 'none',
 'vietnamese',
 'swedish']

In [ ]:
restaurant_id = np.array([x for x in range(500)])
vals = np.random.randint(0,2, (500, 20))
average_rating = (5-1) * np.random.random_sample((500,)) + 1

data = np.concatenate((restaurant_id.reshape(-1,1), average_rating.reshape(-1,1), vals), axis=1)

In [ ]:
item_df = pd.DataFrame(data=data, columns=['id', 'avg_rating']+new_20_features)
item_df.head()

,id,avg_rating,flour based food,belgian,exclusive to eats,burritos,sopas e caldos,local eats,bangladeshi,russian,...,dessert: other,healthy,swedish,middle eastern,chinese: taiwanese,american (traditional),balkan,none,vietnamese,swedish
0,0.0,4.817172,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1.0,1.269477,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,2.0,2.863306,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,3.0,3.486216,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,4.0,2.086762,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [ ]:
item_df.shape

(500, 22)

In [ ]:
num_users = 276
arr = np.array([])

for i in range(num_users):
  n_films = np.random.randint(100,300)
  t = np.concatenate((np.random.randint(0,500,(n_films, 1)),  ((5-1)*np.random.random_sample((n_films, 20))+1) * np.random.randint(0,2,(n_films, 20))),axis=1)
  arr = np.concatenate((arr.reshape(-1,21), t), axis=0)

In [ ]:
user_df = pd.DataFrame(data=arr, columns=['id'] + new_20_features)
user_df.head()

,id,flour based food,belgian,exclusive to eats,burritos,sopas e caldos,local eats,bangladeshi,russian,japanese sweets,...,dessert: other,healthy,swedish,middle eastern,chinese: taiwanese,american (traditional),balkan,none,vietnamese,swedish
0,436.0,0.000000,0.000000,0.000000,2.993807,0.000000,2.121996,0.000000,2.580433,0.00000,...,0.000000,2.038272,2.044336,4.679807,0.000000,3.206972,0.000000,0.000000,3.970496,4.862570
1,266.0,0.000000,2.192443,0.000000,0.000000,2.155206,0.000000,3.942271,4.326943,2.62564,...,0.000000,2.185003,0.000000,0.000000,0.000000,2.003154,2.511042,0.000000,0.000000,0.000000
2,403.0,1.296309,0.000000,0.000000,0.000000,0.000000,4.356457,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,2.980019,0.000000,3.771168,1.222922,0.000000,2.625542
3,125.0,0.000000,0.000000,0.000000,4.684638,0.000000,1.826169,0.000000,0.000000,0.00000,...,2.997541,0.000000,4.383219,3.427981,0.000000,2.447491,0.000000,1.720061,3.453780,0.000000
4,438.0,1.291173,0.000000,4.378789,3.355468,0.000000,3.515908,2.176392,3.816621,0.00000,...,0.000000,0.000000,3.087265,0.000000,4.115836,3.939524,0.000000,1.837269,0.000000,0.000000


In [ ]:
cuisine = pd.merge(item_df, user_df, how='right', suffixes=('', '_remove'), on='id')
cuisine_df = cuisine.drop([i for i in cuisine.columns if 'remove' in i],
               axis=1)
cuisine_df.head()

,id,avg_rating,flour based food,belgian,exclusive to eats,burritos,sopas e caldos,local eats,bangladeshi,russian,...,dessert: other,healthy,swedish,middle eastern,chinese: taiwanese,american (traditional),balkan,none,vietnamese,swedish
0,436.0,2.533726,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
1,266.0,4.821147,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2,403.0,1.378651,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
3,125.0,4.832342,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
4,438.0,2.068612,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [ ]:
cuisine_df[cuisine_df['id'] == 87].head()

,id,avg_rating,flour based food,belgian,exclusive to eats,burritos,sopas e caldos,local eats,bangladeshi,russian,...,dessert: other,healthy,swedish,middle eastern,chinese: taiwanese,american (traditional),balkan,none,vietnamese,swedish
565,87.0,1.464524,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
1765,87.0,1.464524,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
2121,87.0,1.464524,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
2199,87.0,1.464524,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3022,87.0,1.464524,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0


In [ ]:
y_train = ((5 - 1)*np.random.random_sample((cuisine_df.shape[0],1)) + 1) 
y_train[:5] 

array([[2.7217068 ],
       [2.47774065],
       [2.99423965],
       [3.04750591],
       [2.5398792 ]])

In [ ]:
cuisine_df['avg_rating'].head(5)

0    2.533726
1    4.821147
2    1.378651
3    4.832342
4    2.068612
Name: avg_rating, dtype: float64

In [ ]:
print(f"user shape: {user_df.shape} | cuisine shape: {cuisine_df.shape} | y_train shape: {y_train.shape}")

user shape: (56089, 21) | cuisine shape: (56089, 22) | y_train shape: (56089, 1)


In [ ]:
user_df.iloc[:,1:]

,flour based food,belgian,exclusive to eats,burritos,sopas e caldos,local eats,bangladeshi,russian,japanese sweets,australian,dessert: other,healthy,swedish,middle eastern,chinese: taiwanese,american (traditional),balkan,none,vietnamese,swedish
0,0.000000,0.000000,0.000000,2.993807,0.000000,2.121996,0.000000,2.580433,0.000000,1.280426,0.000000,2.038272,2.044336,4.679807,0.000000,3.206972,0.000000,0.000000,3.970496,4.862570
1,0.000000,2.192443,0.000000,0.000000,2.155206,0.000000,3.942271,4.326943,2.625640,3.490217,0.000000,2.185003,0.000000,0.000000,0.000000,2.003154,2.511042,0.000000,0.000000,0.000000
2,1.296309,0.000000,0.000000,0.000000,0.000000,4.356457,0.000000,0.000000,0.000000,2.270476,0.000000,0.000000,0.000000,0.000000,2.980019,0.000000,3.771168,1.222922,0.000000,2.625542
3,0.000000,0.000000,0.000000,4.684638,0.000000,1.826169,0.000000,0.000000,0.000000,0.000000,2.997541,0.000000,4.383219,3.427981,0.000000,2.447491,0.000000,1.720061,3.453780,0.000000
4,1.291173,0.000000,4.378789,3.355468,0.000000,3.515908,2.176392,3.816621,0.000000,1.526248,0.000000,0.000000,3.087265,0.000000,4.115836,3.939524,0.000000,1.837269,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56084,0.000000,4.824558,3.815529,3.307321,4.706346,0.000000,0.000000,3.696451,3.714402,4.418690,2.817563,4.048147,2.469741,0.000000,2.883587,0.000000,0.000000,1.773928,0.000000,3.024711
56085,0.000000,0.000000,2.217283,0.000000,0.000000,2.735233,1.383473,4.628488,2.684027,0.000000,3.135703,3.204309,0.000000,0.000000,1.902833,4.368234,4.166916,4.754663,4.002620,4.444072
56086,4.133678,2.511291,4.965279,1.699039,0.000000,0.000000,2.416953,0.000000,4.648760,4.390596,3.974577,0.000000,3.166109,0.000000,0.000000,0.000000,0.000000,0.000000,3.688840,3.137808
56087,0.000000,0.000000,0.000000,0.000000,1.489424,1.381439,0.000000,0.000000,0.000000,2.403829,0.000000,0.000000,3.265802,0.000000,0.000000,3.687143,0.000000,3.692666,1.696156,4.325988


In [ ]:
# scale training data
item_train_unscaled = cuisine_df
user_train_unscaled = user_df
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(cuisine_df)
item_train = scalerItem.transform(cuisine_df)

scalerUser = StandardScaler()
scalerUser.fit(user_df)
user_train = scalerUser.transform(user_df)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))


In [ ]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"item training data shape: {item_train.shape}")
print(f"item test data shape: {item_test.shape}")

item training data shape: (44871, 22)
item test data shape: (11218, 22)


In [ ]:
num_user_features = user_df.shape[1] - 1
num_item_features = cuisine_df.shape[1] - 1
print(f"user feature: {num_user_features} | item feature: {num_item_features}")

user feature: 20 | item feature: 21


In [ ]:
# MODEL ARCHITECTURE
num_outputs = 16
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
       
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
    
])

item_NN = tf.keras.models.Sequential([
       
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear')
    
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_16 (InputLayer)          [(None, 21)]         0           []                               
                                                                                                  
 sequential_14 (Sequential)     (None, 16)           11984       ['input_15[0][0]']               
                                                                                                  
 sequential_15 (Sequential)     (None, 16)           40592       ['input_16[0][0]']               
                                                                                            

In [ ]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [ ]:
tf.random.set_seed(1)
model.fit([user_train[:,1:], item_train[:,1:]], y_train, epochs=30)

Epoch 1/30
1403/1403 [==============================] - 6s 3ms/step - loss: 0.3377
Epoch 2/30
1403/1403 [==============================] - 6s 4ms/step - loss: 0.3360
Epoch 3/30
1403/1403 [==============================] - 3s 2ms/step - loss: 0.3356
Epoch 4/30
1403/1403 [==============================] - 3s 2ms/step - loss: 0.3355
Epoch 5/30
1403/1403 [==============================] - 4s 3ms/step - loss: 0.3353
Epoch 6/30
1403/1403 [==============================] - 4s 3ms/step - loss: 0.3351
Epoch 7/30
1403/1403 [==============================] - 3s 2ms/step - loss: 0.3348
Epoch 8/30
1403/1403 [==============================] - 3s 2ms/step - loss: 0.3342
Epoch 9/30
1403/1403 [==============================] - 4s 3ms/step - loss: 0.3335
Epoch 10/30
1403/1403 [==============================] - 3s 2ms/step - loss: 0.3327
Epoch 11/30
1403/1403 [==============================] - 3s 2ms/step - loss: 0.3315
Epoch 12/30
1403/1403 [==============================] - 4s 3ms/step - loss: 0.3299
E

In [ ]:
model.evaluate([user_test[:,1:], item_test[:,1:]], y_test)

351/351 [==============================] - 1s 1ms/step - loss: 0.3690


0.3689669966697693

In [ ]:
item_vec = item_df
item_vec.head(5)

,id,avg_rating,flour based food,belgian,exclusive to eats,burritos,sopas e caldos,local eats,bangladeshi,russian,...,dessert: other,healthy,swedish,middle eastern,chinese: taiwanese,american (traditional),balkan,none,vietnamese,swedish
0,0.0,4.817172,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1.0,1.269477,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,2.0,2.863306,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
3,3.0,3.486216,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,4.0,2.086762,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


In [ ]:
user_vec = np.array([0]*21)
user_vec[7] = 5
user_vec[-5] = 0
user_vec = np.tile(user_vec, (500,1))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vec)
sitem_vecs = scalerItem.transform(item_vec)

# make a prediction
y_p = model.predict([suser_vecs[:,1:], sitem_vecs[:,1:]])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]

sorted_index = [str(x) for x in sorted_index[:5]]
sorted_ypu = [str(x) for x in sorted_ypu[:5]]

print(f"user vector: {user_vec[0,1:]}")
print(f"y value: {sorted_ypu}")
ids = item_df.iloc[sorted_index]['id'].tolist()
print(ids)
item_df.iloc[sorted_index]

16/16 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


user vector: [0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0 0]
y value: ['[3.843508]', '[3.8204157]', '[3.8201363]', '[3.786596]', '[3.7785718]']
[317.0, 51.0, 334.0, 71.0, 116.0]


,id,avg_rating,flour based food,belgian,exclusive to eats,burritos,sopas e caldos,local eats,bangladeshi,russian,...,dessert: other,healthy,swedish,middle eastern,chinese: taiwanese,american (traditional),balkan,none,vietnamese,swedish
317,317.0,4.975127,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
51,51.0,3.086608,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
334,334.0,4.469620,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
71,71.0,1.989280,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
116,116.0,3.350692,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
for i in range(100):  
  user_vec = (5-0) * np.random.random_sample((20))
  user_vec = np.tile(user_vec, (500,1))

  # scale our user and item vectors
  suser_vecs = scalerUser.transform(user_vec)
  sitem_vecs = scalerItem.transform(item_vec)

  # make a prediction
  y_p = model.predict([suser_vecs, sitem_vecs])

  # unscale y prediction 
  y_pu = scalerTarget.inverse_transform(y_p)

  # sort the results, highest prediction first
  sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
  sorted_ypu   = y_pu[sorted_index]

  sorted_index = [str(x) for x in sorted_index[:5]]
  sorted_ypu = [str(x) for x in sorted_ypu[:5]]

  with open("index.txt", "a") as f:
    f.write("index: " + ", ".join(sorted_index) + "\n" + "yp: " + ", ".join(sorted_ypu) + "\n\n")
